In [ ]:
# Monte Carlo Portfolio Risk Engine

This notebook demonstrates Monte Carlo VaR calculation and backtesting for portfolio risk management.

## Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from mcrisk import RiskEngine

pd.set_option('display.precision', 4)
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
## Initialize Portfolio

Create a portfolio with SPY (40%), AAPL (30%), and QQQ (30%).


In [ ]:
tickers = ['SPY', 'AAPL', 'QQQ']
weights = np.array([0.4, 0.3, 0.3])

engine = RiskEngine(
    tickers=tickers,
    portfolio_weights=weights,
    lookback=250,
    random_seed=42
)

print(f"Portfolio: {dict(zip(tickers, weights))}")


In [ ]:
## Load Data


In [ ]:
prices = engine.load_data(period="2y")

print(f"Data shape: {prices.shape}")
display(prices.tail())


In [ ]:
## Monte Carlo VaR Analysis

Compare different distribution approaches.


In [ ]:
distributions = ['normal', 't', 'bootstrap']
confidence_levels = [0.95, 0.99]
n_simulations = 10000

results = {}

for dist in distributions:
    print(f"\n{dist.upper()} Distribution:")
    results[dist] = {}
    
    for alpha in confidence_levels:
        var, cvar = engine.compute_mc_var(
            alpha=alpha,
            sims=n_simulations,
            distribution=dist
        )
        results[dist][f'VaR_{int(alpha*100)}%'] = var
        results[dist][f'CVaR_{int(alpha*100)}%'] = cvar

comparison_df = pd.DataFrame(results).T
print("\nResults Comparison:")
display(comparison_df.round(4))


In [ ]:
## Visualizations


In [ ]:
fig = engine.plot_return_distribution(confidence_levels=[0.95, 0.99])
plt.show()

fig = engine.plot_price_paths(n_paths=200)
plt.show()


In [ ]:
## VaR Backtesting


In [ ]:
print("Running VaR backtesting...")

backtest_results = engine.backtest(
    alpha=0.95,
    n_simulations=5000,
    distribution='normal'
)

print(f"Backtest completed with {len(backtest_results)} observations")
display(backtest_results.head())


In [ ]:
summary = engine.backtester.get_backtest_summary()
display(summary)

fig = engine.plot_backtest_results(confidence_level=0.95)
plt.show()
